### CMB Assigment 2 Program ###

In [ ]:
# Program, run imports
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import time
import seaborn as sns
import requests
sns.set(style="darkgrid")

Create Pandas Dataframe

In [ ]:
#df = pd.read_csv("data/21-12-23_complete.csv")
# data_folder = "/Users/Fabian/Desktop/"
data_folder = "/Users/Fabian/Desktop/"

# data_folder = "C:/Users/maxim/Documents/RaspData/Preprocessed Data/"
df = pd.read_csv(data_folder + "1dayAll.csv", encoding = "latin")
#df = pd.read_csv(data_folder + "1A", encoding = "latin")
print(df)
# ['No.', 'Time', 'Source', 'Destination', 'Protocol', 'Length', 'Info', 'dst port']


Filter Local Traffic

In [ ]:
# source and destination should not start with 192.168 to filter local network
filtered = df.loc[~df['Source'].str.startswith("192.168") & df['Destination'].str.startswith("192.168") |
                 df['Source'].str.startswith("192.168") & ~df['Destination'].str.startswith("192.168")]

# filter multicast
filtered = filtered.loc[~filtered['Protocol'].str.contains("MDNS")]
filtered = filtered.loc[filtered['Destination'].str.split('.').str[0].astype(int) < 224]
# filter dhcp
filtered = filtered.loc[~filtered['Destination'].str.contains("192.168.0.1")]
filtered = filtered.loc[~filtered['Destination'].str.contains("192.168.178.1")]
filtered = filtered.loc[~filtered['Destination'].str.contains("0.0.0.0")]

df = filtered.copy()
print(df)

In [ ]:
# helper functions
def utcEntryToTimestamp(entry):
    if '.' in entry:
        row_entry = entry.split(".")[0]
    else:
        row_entry = entry.split(",")[0]
    TIME_FORMAT='%Y-%m-%d %H:%M:%S'
    ts = int(datetime.strptime(row_entry, TIME_FORMAT).timestamp())
    return ts

def utcRowToTimestamp(row):
    return utcEntryToTimestamp(row.at['Time'])
utcRowToTimestamp(df.iloc[0])
# print(df.loc[0].at['Time'])

### Protocol packet distribution ###

In [ ]:
print(df.groupby('Protocol').Source.count())

count_protocol_packets = df.groupby('Protocol').Source.count()

fig, ax = plt.subplots()
plt.title('Protocol Distribution by Amount of Packets')
ax.pie(count_protocol_packets, labels=count_protocol_packets.keys(), autopct='%1.1f%%',)
fig.show()

### Amount of data traffic per protocol ###

In [ ]:
count_protocol_traffic = df.groupby('Protocol').Length.sum()
print(count_protocol_traffic)
fig, ax = plt.subplots()
plt.title('Protocol Distribution by Data Traffic')
ax.pie(count_protocol_traffic, labels=count_protocol_traffic.keys(), autopct='%1.1f%%',)
fig.show()

### Average Data length per Protocol Type ###

In [ ]:
average_protocol_packet_length = df.groupby('Protocol').Length.sum() / df.groupby('Protocol').Length.count()
print(average_protocol_packet_length)
fig, ax = plt.subplots()
ax.tick_params(axis='x', which='major', labelsize=12)
ax.tick_params(axis='x', which='minor', labelsize=12)
plt.xlabel('Protocol')
plt.ylabel('Data [Byte]')
plt.title('Average Data Length per Protocol Type')
ax.bar(average_protocol_packet_length.keys(), average_protocol_packet_length, align='center',)
plt.xticks(rotation=60, ha="right")
fig.show()

### Source addresses ###

In [ ]:
print(df.groupby('Source').Time.count())
print(df.groupby('Destination').Time.count().sort_values())

unique_dests = df['Destination'].unique()

# write ip address destinations to file
file1 = open("Testfile.txt","w")
for row in unique_dests:
    file1.write(row + "\n")
file1.close()

In [ ]:
print(df.groupby('Source').Time.count())
print(df.groupby('Destination').Time.count().sort_values())

### DNS requests ###

In [ ]:
df_dns = df[df['Protocol'].isin(['DNS'])]
df_dns.head()
df_dns_requests_1 = df_dns[df_dns['Info'].str.contains('Standard query')]
# does not contain response
df_dns_requests_2 = df_dns_requests_1[~df_dns_requests_1['Info'].str.contains('response')]
df_dns_resources = df_dns_requests_2['Info'].apply(lambda x: x.split(' ')[-1]).value_counts()
print(df_dns_resources)
# print(df_dns_requests)
# print(df_dns['Info'])


### IP Wordlwide location

In [ ]:
df_destinations =  df['Destination'].unique()

for dest in df_destinations[:10]: # 10 entries limit for now
    if dest.startswith("192.168"):
            continue
    headers = { 'User-Agent': "keycdn-tools:https://www.example.com" }
    url = "https://tools.keycdn.com/geo.json?host={}".format(dest)
    r = requests.get(url, headers=headers)
    print(r.json() )  


### Data traffic over time ###

In [ ]:
def timeMapping(x):
    # include time difference UTC+1
    time = datetime.utcfromtimestamp(x*min_15_duration + 3600)
    if time.minute == 0 and time.hour % 3 == 0:
        return time.strftime("%H:%M")
    else:
        return ""

# find first and last timestamp, then create data structure
min_15_duration = 60 * 15
# df_time_mod = df.copy()
print('Compute intensive task 1/3...')
df['index-time'] = df['Time'] # .apply(lambda x: x)) # map to 15 min window
df['index-time'] = df['index-time'].apply(lambda x: utcEntryToTimestamp(x))

first_entry = df.iloc[0].at['index-time'] // min_15_duration
last_entry = df.iloc[-1].at['index-time'] // min_15_duration

print('from ', df.iloc[0].at['Time'], " to ", df.iloc[-1].at['Time'])

count_packets = np.zeros(last_entry - first_entry + 1)
length_packets = np.zeros(last_entry - first_entry + 1)

x_values_packets = list(range(first_entry, last_entry + 1))
print('Amount of Intervals',len(x_values_packets))

mapping_res = list(map(timeMapping, x_values_packets))

print('Compute intensive task 2/3...')
df['index-time'] = (df['index-time'] // min_15_duration) - first_entry


In [ ]:
count_packets = np.zeros(last_entry - first_entry + 1)
length_packets = np.zeros(last_entry - first_entry + 1)
print('Compute intensive task 3/3...')
time_start = time.time()
for row in df.to_dict('records'):
    count_packets[row['index-time']] += 1
    length_packets[row['index-time']] += row['Length']

# old solution:
#for index, row in df.iterrows():
#    count_packets[row.at['index-time']] += 1
#    length_packets[row.at['index-time']] += row.at['Length']

time_end = time.time()
print(time_end - time_start)

length_packets = length_packets // 1000
#print(count_packets)
#print(length_packets)

fig, ax = plt.subplots()
plt.xlabel('Time')
plt.ylabel('Packets')
plt.title('Packets per timeframe (15 min interval)')
plt.xticks(x_values_packets, mapping_res)
ax.bar(x_values_packets, count_packets, color='black')
fig.show()

fig, ax = plt.subplots()
plt.xlabel('Time')
plt.ylabel('Data [kB]')
plt.title('Data traffic per timeframe (15 min interval)')
plt.xticks(x_values_packets, mapping_res)
ax.bar(x_values_packets, length_packets, color='black')
fig.show()

End of the notebook